## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Hambantota,LK,6.1241,81.1185,77.00,83,13,5.75,few clouds
1,1,Saskylakh,RU,71.9167,114.0833,-33.05,81,83,5.08,broken clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,18.16,79,100,3.44,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,78.30,67,10,11.25,clear sky
4,4,Geraldton,AU,-28.7667,114.6000,69.80,73,75,8.05,broken clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Hambantota,LK,6.1241,81.1185,77.00,83,13,5.75,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,78.30,67,10,11.25,clear sky
9,9,Vaini,TO,-21.2000,-175.2000,75.20,100,75,4.61,moderate rain
10,10,Axim,GH,4.8699,-2.2405,82.76,71,61,11.77,broken clouds
17,17,Champerico,GT,14.3000,-91.9167,82.40,61,0,1.90,clear sky
21,21,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,75,10.36,light rain
23,23,Adrar,MR,20.5022,-10.0711,82.11,12,1,13.80,clear sky
25,25,Montevideo,UY,-34.8335,-56.1674,86.00,48,20,8.05,few clouds
26,26,Grand-Santi,GF,4.2500,-54.3833,82.02,81,30,4.68,scattered clouds
27,27,Kloulklubed,PW,7.0419,134.2556,78.80,94,75,5.75,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Weather Description    231
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Hambantota,LK,77.00,few clouds,6.1241,81.1185,
3,Rikitea,PF,78.30,clear sky,-23.1203,-134.9692,
9,Vaini,TO,75.20,moderate rain,-21.2000,-175.2000,
10,Axim,GH,82.76,broken clouds,4.8699,-2.2405,
17,Champerico,GT,82.40,clear sky,14.3000,-91.9167,
21,Puerto Ayora,EC,77.00,light rain,-0.7393,-90.3518,
23,Adrar,MR,82.11,clear sky,20.5022,-10.0711,
25,Montevideo,UY,86.00,few clouds,-34.8335,-56.1674,
26,Grand-Santi,GF,82.02,scattered clouds,4.2500,-54.3833,
27,Kloulklubed,PW,78.80,broken clouds,7.0419,134.2556,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != '']

In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))